## POC for calculating and wavering interest for credit-card account

GitHub: https://github.com/MkershMambu/MambuAPINotebook.git

Viewer: https://nbviewer.jupyter.org/github/MkershMambu/MambuAPINotebook/blob/master/MambuAPINotebook.ipynb

In [1]:
from IPython.display import HTML, display
import tabulate
import datetime


In [95]:
def resetBalances():
    global cc_balances
    cc_balances = {
        "currentPeriod" : {
            "interest":0,
            "principal":0,
        },
        "currentBillPeriod" : {
            "fullGraceEnabled": True,
            "isActive": False,
            "interest":0,
            "interestBilled":0,
            "principal":0,
            "principalBilled":0,
            "paymentDueDate": None
        },
        "historicPeriods" : {
            "interest":0,
            "principal":0
        },
        'event_history':[],
        "extraCreditBalance": 0
    }

In [120]:
def accrueInterest(obj,numDays, interestRate):
    # print("number days to accrue interest", numDays)
    interest = obj["interest"]
    principal = obj["principal"]
    interest += (principal * interestRate) * numDays
    obj["interest"] = interest

def repaymentTop(amount):
    global cc_balances
    # Balance left over from previous repayment
    extraCreditBalance = cc_balances["extraCreditBalance"]
    amount += extraCreditBalance
    
    if cc_balances["currentBillPeriod"]["isActive"] == True:
        leftOverAmount = repayment(cc_balances["currentBillPeriod"],amount, True)
        # print("leftOverAmount Bill", leftOverAmount)
    else:
        leftOverAmount = repayment(cc_balances["historicPeriods"],amount, False, isHistoric=True)
        # print("leftOverAmount historic", leftOverAmount)
            
    if leftOverAmount > 0:
        leftOverAmount = repayment(cc_balances["currentPeriod"],leftOverAmount, False) 
        # print("leftOverAmount current", leftOverAmount)
        
    if leftOverAmount > 0:
        extraCreditBalance = cc_balances["extraCreditBalance"]
        extraCreditBalance += leftOverAmount
        cc_balances["extraCreditBalance"] = extraCreditBalance
    else:
        cc_balances["extraCreditBalance"] = 0
            
def repayment(obj,amount, isBill, isHistoric=False):
    principal = obj["principal"]
    interestAccrued = obj["interest"]
    interestBilled = obj.get("interestBilled", interestAccrued)
    interestPaid = obj.get("interestPaid", 0)
    interestRemain = interestBilled - interestPaid
    
    leftOver = interestRemain - amount
    if leftOver < 0:
        interestPaid = interestBilled
        leftOver = principal + leftOver
        if leftOver < 0:
            principal = 0
        else:
            principal = leftOver
            leftOver = 0
    else:
        interestPaid = leftOver
        leftOver = 0
    obj["principal"] = principal
    if isHistoric == False:
        obj["interestPaid"] = interestPaid
    else:
        print("Historic interest paid", interestPaid)
        obj["interest"] -= interestPaid
    return -leftOver

def endBillPeriod():
    global cc_balances
    # print("EndBill")
    
    fullGraceEnabled = cc_balances["currentBillPeriod"]["fullGraceEnabled"]
    billInterest = cc_balances["currentBillPeriod"]["interest"]
    billInterestBilled = cc_balances["currentBillPeriod"]["interestBilled"]
    billPrincipal = cc_balances["currentBillPeriod"]["principal"]
    billPrincipalBilled = cc_balances["currentBillPeriod"]["principalBilled"]
    billInterestPaid = cc_balances["currentBillPeriod"].get("interestPaid",0)
    amountPaid = (billPrincipalBilled - billPrincipal + billInterestPaid)
    
    # determine whether to waver interest
    if fullGraceEnabled and (amountPaid >= billPrincipalBilled):
        # Full Waver applies
        # print("Full Waver applied", amountPaid, billPrincipalBilled)
        newHistoricInterest = 0
        newHistoricPrincipal = 0
        extraCredit = -(billPrincipal-billInterestPaid)
    else:
        # 1) calculate %bill paid:
        #    percentageBillPaid = amount_paid / (principleBilled+InterestBilled)
        percentageBillPaid =  amountPaid / (billPrincipalBilled + billInterestBilled)
        # print("Percentage of Bill Paid = ", amountPaid, percentageBillPaid)
        # 3) If percentageBillPaid = 100% then marked Bill and Historic as paid
        if (percentageBillPaid >= 1):
            # print("100% Bill paid")
            cc_balances["currentBillPeriod"]["fullGraceEnabled"] = True
            newHistoricInterest = 0
            newHistoricPrincipal = 0 
        else:
            # 4) If <100% then Interest_owed = Interest_accrued_Billed + (Interest_accrued * Bill_percentage_paid)
            cc_balances["currentBillPeriod"]["fullGraceEnabled"] = False
            newHistoricInterest = (1 - percentageBillPaid)*(billInterest-billInterestPaid)
            newHistoricPrincipal = billPrincipal
    
    cc_balances["historicPeriods"]["interest"] = newHistoricInterest 
    cc_balances["historicPeriods"]["principal"] = newHistoricPrincipal 
    cc_balances["currentBillPeriod"]["interest"] = 0
    cc_balances["currentBillPeriod"]["interestBilled"] = 0
    cc_balances["currentBillPeriod"]["interestPaid"] = 0
    cc_balances["currentBillPeriod"]["principal"] = 0
    cc_balances["currentBillPeriod"]["principalBilled"] = 0
    cc_balances["currentBillPeriod"]["isActive"] = False
    cc_balances["currentBillPeriod"]["paymentDueDate"] = None

def applyInterest(event, fromDate, toDate, date="", desc=""):
    date_time_str = fromDate
    date_time_str2 = toDate
    date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%d')
    date_time_obj2 = datetime.datetime.strptime(date_time_str2, '%Y-%m-%d')
    delta = (date_time_obj2 - date_time_obj).days 
    applyEvent(event, date=date, desc=desc, numDays=delta)

def applyEvent(event, date="", desc="", amount=0, numDays=0):
    global cc_balances
    
    if amount==0:
        amount = event.get("amount",0)
    if desc=="":
        desc = event.get("desc","")
    if desc=="":
        if amount > 0:
            desc = "amount="+str(amount)
    if numDays==0:
        numDays = event.get("numDays",0)
    if numDays>0:
        desc = "days="+str(numDays)
        
    if (event["type"] == "bill"):
        cc_balances["currentBillPeriod"]["isActive"] = True
        cc_balances["currentBillPeriod"]["paymentDueDate"] = event["paymentDueDate"]
        historic_interest = cc_balances["historicPeriods"]["interest"]
        historic_principal = cc_balances["historicPeriods"]["principal"]
        current_interest = cc_balances["currentPeriod"]["interest"]
        current_interestPaid = cc_balances["currentPeriod"].get("interestPaid",0)
        current_principal = cc_balances["currentPeriod"]["principal"]
        if (historic_principal+historic_interest) > 0:
            cc_balances["currentBillPeriod"]["fullInterestGrace"] = False
        else:
            cc_balances["currentBillPeriod"]["fullInterestGrace"] = True
        cc_balances["currentBillPeriod"]["interestBilled"] = historic_interest + current_interest
        cc_balances["currentBillPeriod"]["interest"] = historic_interest + current_interest
        cc_balances["currentBillPeriod"]["interestPaid"] = current_interestPaid
        cc_balances["currentBillPeriod"]["principal"] = historic_principal + current_principal
        cc_balances["currentBillPeriod"]["principalBilled"] = cc_balances["currentBillPeriod"]["principal"]
        cc_balances["currentPeriod"]["interest"] = 0
        cc_balances["currentPeriod"]["interestPaid"] = 0
        cc_balances["currentPeriod"]["principal"] = 0
    elif (event["type"] == "purchase"):
        principal = cc_balances['currentPeriod']["principal"]
        principal += amount
        cc_balances['currentPeriod']["principal"] = principal
    elif (event["type"] == "interestAccrue"):
        # print("interestAccrue")
        num_days = numDays
        interestRatePerDay = event["interestPerDay"]
        accrueInterest(cc_balances['currentPeriod'], num_days, interestRatePerDay)
        if cc_balances["currentBillPeriod"]["isActive"] == True:
            accrueInterest(cc_balances['currentBillPeriod'], num_days, interestRatePerDay)
        else:
            accrueInterest(cc_balances['historicPeriods'], num_days, interestRatePerDay)
    elif (event["type"] == "endBill"):
        endBillPeriod()
    elif (event["type"] == "repayment"):
        repaymentTop(amount)
            
    logEvent = {
        "date": date,
        "desc": desc,
        "event": event,
        "currentPeriod" : {
            "interest": cc_balances['currentPeriod']["interest"],
            "interestPaid": cc_balances['currentPeriod'].get("interestPaid",0),
            "principal": cc_balances['currentPeriod']["principal"],
        },
        "currentBillPeriod" : {
            "isActive": cc_balances['currentBillPeriod']["isActive"],
            "interest": cc_balances['currentBillPeriod']["interest"],
            "interestBilled": cc_balances['currentBillPeriod']["interestBilled"],
            "interestPaid": cc_balances['currentBillPeriod'].get("interestPaid",0),
            "principal": cc_balances['currentBillPeriod']["principal"],
            "principalBilled": cc_balances['currentBillPeriod']["principalBilled"],
            "paymentDueDate": cc_balances['currentBillPeriod']["paymentDueDate"]
        },
        "historicPeriods" : {
            "interest": cc_balances['historicPeriods']["interest"],
            "principal":cc_balances['historicPeriods']["principal"]
        },
        
    }        
    cc_balances["event_history"].append(logEvent)
        
    
    

In [4]:
event1 = {
    "type": "bill",
    "paymentDueDate": "01/01/2020"
}

In [5]:
event2 = {
    "type": "purchase",
    "description": "Waitrose bill",
    "amount": 100.00
}

In [6]:
event3 = {
    "type": "interestAccrue",
    "numDays": 1,
    "interestPerDay": 0.01
}

In [7]:
event4 = {
    "type": "endBill",
}

In [8]:
event5 = {
    "type": "repayment",
    "amount": 100.00
}

In [9]:
def showHistory():
    global cc_balances
    evCount = 1
    historyTable = [["#", "Date", "Type", "Description", "Principal(Curr)","Interest Paid(Current)", "Interest(Current)", "Principal(Bill)", "Principle Billed", "Interest Billed","Interest Paid(Bill)", "Interest(Bill)", "Principal(Hist)","Interest(Hist)"]]
    for eventItem in cc_balances["event_history"]:
        evDate = eventItem["date"]
        evType = eventItem["event"]["type"]
        evDesc = eventItem["desc"]
        prinCurr = round(eventItem["currentPeriod"]["principal"],2)
        intCurr = round(eventItem["currentPeriod"]["interest"],2)
        intPaidCurr = round(eventItem["currentPeriod"].get("interestPaid",0),2)
        prinBill = round(eventItem["currentBillPeriod"]["principal"],2)
        prinBilled = round(eventItem["currentBillPeriod"]["principalBilled"],2)
        intBill = round(eventItem["currentBillPeriod"]["interest"],2)
        intBilled = round(eventItem["currentBillPeriod"]["interestBilled"],2)
        intPaidBill = round(eventItem["currentBillPeriod"].get("interestPaid",0),2)
        prinHist = round(eventItem["historicPeriods"]["principal"],2)
        intHist = round(eventItem["historicPeriods"]["interest"],2)
        historyItem = []
        historyItem.extend([evCount,evDate, evType,evDesc,prinCurr, intPaidCurr, intCurr,prinBill, prinBilled, intBilled, intPaidBill, intBill,prinHist,intHist])
        historyTable.append(historyItem)
        evCount +=1
    display(HTML(tabulate.tabulate(historyTable, tablefmt='html')))
    print("Extra Credit Balance:", cc_balances["extraCreditBalance"])

In [123]:
# Test Full waver
resetBalances()
applyEvent(event2,date="") #purchase
applyEvent(event3, numDays=2) #accrue
applyEvent(event1) #bill
applyEvent(event5,amount=100) # repayment
applyEvent(event4) #endBill

showHistory()

#,Date,Type,Description,Principal(Curr),Interest Paid(Current),Interest(Current),Principal(Bill),Principle Billed,Interest Billed,Interest Paid(Bill),Interest(Bill),Principal(Hist),Interest(Hist)
1,,purchase,amount=100.0,100.0,0,0,0,0,0,0,0,0,0
2,,interestAccrue,days=2,100.0,0,2.0,0,0,0,0,0,0,0.0
3,,bill,,0,0,0,100.0,100.0,2.0,0,2.0,0,0.0
4,,repayment,amount=100,0,0,0,2.0,100.0,2.0,2.0,2.0,0,0.0
5,,endBill,,0,0,0,0,0,0,0,0,0,0


Extra Credit Balance: 0


In [108]:
# Test Not fullgrace waver - when outstanding Historic balances
resetBalances()
# Create some historic dept
applyEvent(event2,date="") #purchase
applyEvent(event1) #bill
applyEvent(event4) #endBill
# Show that for the next bill fullgrace is not available
applyEvent(event2,date="") #purchase
applyEvent(event3, numDays=2) #accrue
applyEvent(event1) #bill
# Pay the bill+Interest in full so that fullgrace applies next bill 
applyEvent(event5,amount=204) # repayment
applyEvent(event4) #endBill
# Show that for next period fullgrace works
applyEvent(event2,date="") #purchase
applyEvent(event3, numDays=2) #accrue
applyEvent(event1) #bill
applyEvent(event5,amount=100)
applyEvent(event4) #endBill
showHistory()

#,Date,Type,Description,Principal(Curr),Interest Paid(Current),Interest(Current),Principal(Bill),Principle Billed,Interest Billed,Interest Paid(Bill),Interest(Bill),Principal(Hist),Interest(Hist)
1,,purchase,amount=100.0,100.0,0,0,0,0,0,0,0,0,0
2,,bill,,0,0,0,100.0,100.0,0,0,0,0,0
3,,endBill,,0,0,0,0,0,0,0,0,100.0,0.0
4,,purchase,amount=100.0,100.0,0,0,0,0,0,0,0,100.0,0.0
5,,interestAccrue,days=2,100.0,0,2.0,0,0,0,0,0,100.0,2.0
6,,bill,,0,0,0,200.0,200.0,4.0,0,4.0,100.0,2.0
7,,repayment,amount=204,0,0,0,0.0,200.0,4.0,4.0,4.0,100.0,2.0
8,,endBill,,0,0,0,0,0,0,0,0,0,0
9,,purchase,amount=100.0,100.0,0,0,0,0,0,0,0,0,0
10,,interestAccrue,days=2,100.0,0,2.0,0,0,0,0,0,0,0.0


Extra Credit Balance: 0


In [124]:
# Test Partial waver
resetBalances()
applyEvent(event2,date="01/01/20", desc="£100") #purchase
applyEvent(event3) #accrue
applyEvent(event1) #bill
applyEvent(event5,amount=99) # repayment
applyEvent(event4) #endBill

showHistory()

#,Date,Type,Description,Principal(Curr),Interest Paid(Current),Interest(Current),Principal(Bill),Principle Billed,Interest Billed,Interest Paid(Bill),Interest(Bill),Principal(Hist),Interest(Hist)
1,01/01/20,purchase,£100,100.0,0,0,0,0,0,0,0,0,0
2,,interestAccrue,days=1,100.0,0,1.0,0,0,0,0,0,0,0.0
3,,bill,,0,0,0,100.0,100.0,1.0,0,1.0,0,0.0
4,,repayment,amount=99,0,0,0,2.0,100.0,1.0,1.0,1.0,0,0.0
5,,endBill,,0,0,0,0,0,0,0,0,2.0,0.0


Extra Credit Balance: 0


In [125]:
# Test Partial waver - With accrual of Bill interest
resetBalances()
applyEvent(event2,date="01/01/20") #purchase
applyEvent(event3) #accrue
applyEvent(event1) #bill
applyEvent(event3) #accrue
applyEvent(event5,amount=99) # repayment
applyEvent(event4) #endBill

showHistory()

#,Date,Type,Description,Principal(Curr),Interest Paid(Current),Interest(Current),Principal(Bill),Principle Billed,Interest Billed,Interest Paid(Bill),Interest(Bill),Principal(Hist),Interest(Hist)
1,01/01/20,purchase,amount=100.0,100.0,0,0,0,0,0,0,0,0,0
2,,interestAccrue,days=1,100.0,0,1.0,0,0,0,0,0,0,0.0
3,,bill,,0,0,0,100.0,100.0,1.0,0,1.0,0,0.0
4,,interestAccrue,days=1,0,0,0.0,100.0,100.0,1.0,0,2.0,0,0.0
5,,repayment,amount=99,0,0,0.0,2.0,100.0,1.0,1.0,2.0,0,0.0
6,,endBill,,0,0,0.0,0,0,0,0,0,2.0,0.02


Extra Credit Balance: 0


In [126]:
# Test Full waver - With extra current transactions
resetBalances()
applyEvent(event2,date="01/01/20") #purchase
applyEvent(event3) #accrue
applyEvent(event1) #bill
applyEvent(event2,date="05/01/20")
applyEvent(event5,amount=100) # repayment
applyEvent(event4) #endBill
showHistory()

#,Date,Type,Description,Principal(Curr),Interest Paid(Current),Interest(Current),Principal(Bill),Principle Billed,Interest Billed,Interest Paid(Bill),Interest(Bill),Principal(Hist),Interest(Hist)
1,01/01/20,purchase,amount=100.0,100.0,0,0,0,0,0,0,0,0,0
2,,interestAccrue,days=1,100.0,0,1.0,0,0,0,0,0,0,0.0
3,,bill,,0,0,0,100.0,100.0,1.0,0,1.0,0,0.0
4,05/01/20,purchase,amount=100.0,100.0,0,0,100.0,100.0,1.0,0,1.0,0,0.0
5,,repayment,amount=100,100.0,0,0,1.0,100.0,1.0,1.0,1.0,0,0.0
6,,endBill,,100.0,0,0,0,0,0,0,0,0,0


Extra Credit Balance: 0


In [65]:
# Test Interest Paid against current transactions
resetBalances()
applyEvent(event2,date="01/01/20") #purchase
applyEvent(event3) #accrue
applyEvent(event5,amount=100) # repayment
applyEvent(event3) #accrue
applyEvent(event5,amount=1)
applyEvent(event3) #accrue
applyEvent(event1) # Bill
applyEvent(event4) #endBill
showHistory()

#,Date,Type,Description,Principal(Curr),Interest Paid(Current),Interest(Current),Principal(Bill),Principle Billed,Interest Billed,Interest Paid(Bill),Interest(Bill),Principal(Hist),Interest(Hist)
1,01/01/20,purchase,amount=100.0,100.0,0,0,0,0,0,0,0,0,0
2,,interestAccrue,days=1,100.0,0,1.0,0,0,0,0,0,0,0.0
3,,repayment,amount=100,1.0,1.0,1.0,0,0,0,0,0,0,0.0
4,,interestAccrue,days=1,1.0,1.0,1.01,0,0,0,0,0,0,0.0
5,,repayment,amount=1,0.01,1.01,1.01,0,0,0,0,0,0,0.0
6,,interestAccrue,days=1,0.01,1.01,1.01,0,0,0,0,0,0,0.0
7,,bill,,0,0,0,0.01,0.01,1.01,1.01,1.01,0,0.0
8,,endBill,,0,0,0,0,0,0,0,0,0,0
9,01/01/20,purchase,amount=33.5,33.5,0,0,0,0,0,0,0,0,0


Extra Credit Balance: 0


In [121]:
# Template Billing cycle for a credit card - Billing Cycle Periods (10th-10th), Payment Due (1st)
resetBalances()
applyEvent(event2,date="11/12/19") #purchase
applyInterest(event3,fromDate="2019-12-11", toDate="2019-12-13")
applyEvent(event2,date="13/12/19") #purchase
applyInterest(event3,fromDate="2019-12-13", toDate="2020-01-01")
applyEvent(event4,date="01/01/20") #endBill
applyInterest(event3,fromDate="2020-01-01", toDate="2020-01-10")
applyEvent(event1,date="10/01/20") # Bill
applyInterest(event3,fromDate="2020-01-10", toDate="2020-02-01")
applyEvent(event4,date="01/02/20") #endBill
applyInterest(event3,fromDate="2020-02-01", toDate="2020-02-10")
applyEvent(event1,date="10/02/20") # Bill
applyInterest(event3,fromDate="2020-02-10", toDate="2020-03-01")
applyEvent(event4,date="01/03/20") #endBill
applyInterest(event3,fromDate="2020-03-01", toDate="2020-03-10")
applyEvent(event1,date="10/03/20") # Bill
applyInterest(event3,fromDate="2020-03-10", toDate="2020-04-01")
applyEvent(event4,date="01/04/20") #endBill
applyInterest(event3,fromDate="2020-04-01", toDate="2020-04-10")
applyEvent(event1,date="10/04/20") # Bill
applyInterest(event3,fromDate="2020-04-10", toDate="2020-05-01")
applyEvent(event4,date="01/05/20") #endBill
applyEvent(event5,amount=500) # repayment
applyEvent(event1,date="03/05/20") # Bill
showHistory()

Historic interest paid 282.0


#,Date,Type,Description,Principal(Curr),Interest Paid(Current),Interest(Current),Principal(Bill),Principle Billed,Interest Billed,Interest Paid(Bill),Interest(Bill),Principal(Hist),Interest(Hist)
1,11/12/19,purchase,amount=100.0,100.0,0,0,0,0,0,0,0,0,0
2,,interestAccrue,days=2,100.0,0,2.0,0,0,0,0,0,0,0.0
3,13/12/19,purchase,amount=100.0,200.0,0,2.0,0,0,0,0,0,0,0.0
4,,interestAccrue,days=19,200.0,0,40.0,0,0,0,0,0,0,0.0
5,01/01/20,endBill,,200.0,0,40.0,0,0,0,0,0,0,0
6,,interestAccrue,days=9,200.0,0,58.0,0,0,0,0,0,0,0.0
7,10/01/20,bill,,0,0,0,200.0,200.0,58.0,0,58.0,0,0.0
8,,interestAccrue,days=22,0,0,0.0,200.0,200.0,58.0,0,102.0,0,0.0
9,01/02/20,endBill,,0,0,0.0,0,0,0,0,0,200.0,102.0
10,,interestAccrue,days=9,0,0,0.0,0,0,0,0,0,200.0,120.0


Extra Credit Balance: 18.0
